In [17]:
#import required libraries
import numpy as np
from numpy import array, hstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

## Univariate LSTM 


In [2]:
#Define function to split a univariate sequence
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        #find the end of this pattern
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        #gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
#define input sequence
raw_seq = [10,20,30,40,50,60,70,80,90,100]
#choose a number of time steps
n_steps = 3
#split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90
[70 80 90] 100


In [4]:
#reshape from [sample, timestamps] into [samples, timestamps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
X.shape

(7, 3, 1)

In [5]:
#define the model LSTM
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 40901 (159.77 KB)
Trainable params: 40901 (159.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
#fit model
model.fit(X, y, epochs=10, verbose=0)

In [8]:
# demonstrate prediction
x_input = array([70, 80, 90])
X_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(X_input, verbose=0)
print(yhat)

[[9.538371]]


## Multivariate LSTM

In [9]:
#define function to split a multiple input sequence
def split_sequence1(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        #find the end of this pattern
        end_ix = i + n_steps
        #check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        #gather input and output parts of the patterns
        seq_x, seq_y = sequence[i:end_ix,:-1], sequence[end_ix-1,-1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [19]:
#define input sequence
in_seq1 = array([10,20,30,40,50,60,70,80,90,100])
in_seq2 = array([5,10.15,20,25,30,35,40,45,50])
in_seq3 = array([30,50,70,90,110,130,150,170,190,210])
#convert to [rows, columns] structure
max_length = max(len(in_seq1), len(in_seq2), len(in_seq3))

# Pad the shorter sequences to match the maximum length
in_seq1 = np.pad(in_seq1, (0, max_length - len(in_seq1)), mode='constant')
in_seq2 = np.pad(in_seq2, (0, max_length - len(in_seq2)), mode='constant')
in_seq3 = np.pad(in_seq3, (0, max_length - len(in_seq3)), mode='constant')

# Convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
in_seq3 = in_seq3.reshape((len(in_seq3), 1))
#horizontally stack columns
dataset = np.hstack((in_seq1, in_seq2, in_seq3))
dataset

array([[ 10.  ,   5.  ,  30.  ],
       [ 20.  ,  10.15,  50.  ],
       [ 30.  ,  20.  ,  70.  ],
       [ 40.  ,  25.  ,  90.  ],
       [ 50.  ,  30.  , 110.  ],
       [ 60.  ,  35.  , 130.  ],
       [ 70.  ,  40.  , 150.  ],
       [ 80.  ,  45.  , 170.  ],
       [ 90.  ,  50.  , 190.  ],
       [100.  ,   0.  , 210.  ]])

In [20]:
#choose a number of time steps
n_steps = 3
#convert into input/output
X, y = split_sequence1(dataset, n_steps)
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])

(7, 3, 2) (7,)
[[10.    5.  ]
 [20.   10.15]
 [30.   20.  ]] 70.0
[[20.   10.15]
 [30.   20.  ]
 [40.   25.  ]] 90.0
[[30. 20.]
 [40. 25.]
 [50. 30.]] 110.0
[[40. 25.]
 [50. 30.]
 [60. 35.]] 130.0
[[50. 30.]
 [60. 35.]
 [70. 40.]] 150.0
[[60. 35.]
 [70. 40.]
 [80. 45.]] 170.0
[[70. 40.]
 [80. 45.]
 [90. 50.]] 190.0


In [22]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 2
X = X.reshape((X.shape[0], X.shape[1], n_features))
X.shape

(7, 3, 2)

In [23]:
#define model (vanillia LSTM)
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 100)               41200     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 41301 (161.33 KB)
Trainable params: 41301 (161.33 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model.fit(X, y, epochs=100, verbose=0)

In [26]:
#demonstrate prediction
x_input = array([[70,35],[80,40],[90,45]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[184.90744]]
